# Vision-in-the-Loop Embodied UI Agent
## Redistricting Example
#### Dr John Hughes

In [ ]:
# Allow "from src...." imports when running this notebook
import os, sys
sys.path.append(os.path.abspath("."))

## Example Small ReAct-Style Loop 
#### Demonstrates capabilities of natural language querying of visual data
- Sends a map screenshot or PDF page to LLaMA 3.2-Vision in Ollama
- Asks for structured JSON output describing districts
- Asks for a plain summary focusing on the referenced region

##### ReAct Loop
- Vision Language Model parses visual data → you refine with natural language commands → the model reasons again
- CAn add calls to external tools such as GEoPandas or QGIS for visualization

In [ ]:
import ollama
from PIL import Image

# Step 1 - send an image of a redistricting map
response = ollama.chat(
    model="llama3.2-vision",
    messages=[{
        "role": "user",
        "content": (
            "You are an assistant that reads redistricting maps. "
            "Extract any district names, numbers, and boundary notes. "
            "Output in strict JSON format with keys: district, description."
        ),
        "images": ["images/redistricting_map.png"]  # path to local image
    }]
)

print("Raw Model Output:\n", response["message"]["content"])

# Step 2 - feed that back for regional focus
structured_data = response["message"]["content"]

follow_up = ollama.chat(
    model="llama3.2-vision",
    messages=[{
        "role": "user",
        "content": (
            f"Here is extracted redistricting data:\n{structured_data}\n\n"
            "Summarize only the impact on Glendale and Burbank communities."
        )
    }]
)

print("\nRegional Summary:\n", follow_up["message"]["content"])